In [1]:
val bucketname = "unibo-bd2223-paolopenazzi"

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
6,application_1675774464703_0007,spark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

bucketname: String = unibo-bd2223-paolopenazzi


# DataPreparation

In [12]:
case class FlightData(
    flightID:String,
    searchDate:String,
    searchMonth:String,
    searchDay:String,
    flightDate:String,
    flightMonth:String,
    flightDay:String,
    startingAirport:String,
    destinationAirport:String,
    duration:Int,
    isEconomy:Boolean,
    isRefundable:Boolean,
    isNonStop:Boolean,
    baseFare:Double,
    totalFare:Double,
    seatsRemaining:Int,
    travelDistance:Int
)

object FlightData {
    def parse(line:String) = {
        val input = line.split(",")
        val flightID = input(0)
        val searchDate = input(1)
        val searchMonth = searchDate.substring(5,7)
        val searchDay = searchDate.substring(8,10)
        val flightDate = input(2)
        val flightMonth = flightDate.substring(5,7)
        val flightDay = flightDate.substring(8,10)
        val startingAirport = input(3)
        val destinationAirport = input(4)
        val dur = input(6).replace("P","").replace("T","").split("D|H|M")
        val duration = durationArray.length match {
            case 3 => dur(0) * 1440 + dur(1) * 60 + dur(2)
            case 2 => 
        }
        val isEconomy = input(8).toBoolean
        val isRefundable = input(9).toBoolean
        val isNonStop = input(10).toBoolean
        val baseFare = input(11).toDouble
        val totalFare = input(12).toDouble
        val seatsRemaining = input(13).toInt
        val travelDistance = input(14) match {
            case "" => 0
            case _ => input(14).toInt
        }
        new FlightData(flightID,searchDate,searchMonth,searchDay,flightDate,flightMonth,flightDay,startingAirport,
                       destinationAirport,duration,isEconomy,isRefundable,isNonStop,baseFare,totalFare,
                       seatsRemaining,travelDistance)
    }
}

val rdd = sc.textFile("s3a://"+bucketname+"/datasets/itineraries.csv")
val header = rdd.first(); //extract header
val rddFlights = rdd.filter(row => row != header).map(FlightData.parse)
rddFlights.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 13.0 failed 4 times, most recent failure: Lost task 3.3 in stage 13.0 (TID 25, ip-172-31-82-188.ec2.internal, executor 4): java.lang.NumberFormatException: For input string: "P1DT1"
	at java.lang.NumberFormatException.forInputString(NumberFormatException.java:65)
	at java.lang.Integer.parseInt(Integer.java:580)
	at java.lang.Integer.parseInt(Integer.java:615)
	at scala.collection.immutable.StringLike$class.toInt(StringLike.scala:273)
	at scala.collection.immutable.StringOps.toInt(StringOps.scala:29)
	at FlightData$.parse(<console>:39)
	at $anonfun$2.apply(<console>:29)
	at $anonfun$2.apply(<console>:29)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:410)
	at org.apache.spark.util.Utils$.getIteratorSize(Utils.scala:1864)
	at org.apache.spark.rdd.RDD$$anonfun$count$1.apply(RDD.scala:1213)
	at org.apache.spark.rdd.RDD$$anonfun$count$1.apply(RDD.scala:1213)
	at org.apach

In [13]:
val rddTest = sc.textFile("s3a://"+bucketname+"/datasets/itineraries.csv")
val header = rddTest.first(); //extract header
val rddA = rddTest.filter(row => row != header).filter(x => x.contains("P1DT1"))
rddA.take(1000).last

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rddTest: org.apache.spark.rdd.RDD[String] = s3a://unibo-bd2223-paolopenazzi/datasets/itineraries.csv MapPartitionsRDD[39] at textFile at <console>:27
header: String = legId,searchDate,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,elapsedDays,isBasicEconomy,isRefundable,isNonStop,baseFare,totalFare,seatsRemaining,totalTravelDistance,segmentsDepartureTimeEpochSeconds,segmentsDepartureTimeRaw,segmentsArrivalTimeEpochSeconds,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode
rddA: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[41] at filter at <console>:27
res6: String = 6e643b6e16e47139393c833ec90c686b,2022-05-21,2022-07-04,OAK,DEN,XAVNA0ME,P1DT1H27M,1,False,False,False,244.66,292.21,9,,1656939600||1657026000,2022-07-04T06:00:00.000-07:00||2022-07-05T07:00:00.000-06:00,1656946380||1657031220,2022-07-

In [14]:
val test = "P1DT1H27M"
val array = test.replace("P","").replace("T","").split("D|H|M")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

test: String = P1DT1H27M
array: Array[String] = Array(1, 1, 27)


In [ ]:
import spark.implicits._
val columns = Seq("flightID","searchDate","searchMonth","searchDay","flightDate","flightMonth","flightDay","startingAirport","destinationAirport",
     "duration","isEconomy","isRefundable","isNonStop","baseFare","totalFare","seatsRemaining","travelDistance")
val flightDataframe = rddFlights.toDF(columns:_*)
flightDataframe.show(60,truncate=40,vertical=true)

In [ ]:
import java.time.format.DateTimeFormatter
import java.time.LocalDateTime
import java.time.Duration
import java.time.temporal.ChronoUnit.DAYS
import java.time.LocalDate

type rddType = (String,String,String,String,String,String,String,String,String,Int,Boolean,Boolean,Boolean,Double,Double,Int,Int)

def enoughData(x: rddType): Long = {
        val searchDate = LocalDate.parse(x._2);
        val flightDate = LocalDate.parse(x._5);
        val daysBetween = DAYS.between(searchDate, flightDate);
        daysBetween
}

In [ ]:
val rddFiltered = rddFlights.filter(x => x.searchDate != "" && x.flightDate != "")
rddFiltered.count()

In [ ]:
import java.time.format.DateTimeFormatter
import java.time.LocalDateTime
import java.time.Duration
import java.time.temporal.ChronoUnit.DAYS
import java.time.LocalDate

rddFlights.filter(x => DAYS.between(LocalDate.parse(x.searchDate), LocalDate.parse(x.flightDate)) > 7).collect()

In [ ]:
val filtered = rddFlightsParsed.filter(x => x._2.equals(""))
filtered.take(1)


In [ ]:
rddFlights.take(2)

In [ ]:

val testA = "2022-04-16"
val testB = "2022-05-16"
val dtf = DateTimeFormatter.ofPattern("yyyy-MM-dd");
val A = LocalDate.parse(testA, dtf);
val B = LocalDate.parse(testB, dtf);
val daysBetween = DAYS.between(A, B);
daysBetween > 7